<a href="https://colab.research.google.com/github/Gajendra-theDataEngineer004627/BigDataAssignments/blob/Assignment11/Assignment12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update -y

In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [4]:
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [5]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [7]:
!pip install -q findspark
import findspark
findspark.init()

In [8]:
from pyspark.sql import SparkSession
spark = (SparkSession
 .builder
 .appName("<app_name>")
 .getOrCreate())

In [ ]:
# function to doanload data files from the google drive
import gdown
def downloadfiles_fromFolder() :
    url = "https://drive.google.com/drive/folders/1Hf8PijpBSNyDjwr-YgozGDRL8UqMDq3D?usp=sharing"   # can also be used input()
    if url.split("/")[-1]== "?usp=sharing":
       url = url.replace("?usp=sharing","")
    gdown.download_folder(url, output="/content")

downloadfiles_fromFolder()

In [23]:
# loading the data in dataframe and then validate accordingly

deptDf = spark.read.format("json").option("path","/content/dept.json").load()
empDf = spark.read.format("json").option("path","/content/employee.json").load()

deptDf1 = deptDf.withColumnRenamed("deptid","deptid_dept")
empDf1 = empDf.withColumnRenamed("deptid","deptid_emp")
deptDf1.show()
empDf1.show(truncate =False)


+---------+-----------+
| deptName|deptid_dept|
+---------+-----------+
|       IT|         11|
|       HR|         21|
|Marketing|         31|
|      Fin|         41|
|    Admin|         51|
+---------+-----------+

+---------------------+---+----------+-------+----+------+
|address              |age|deptid_emp|empname|id  |salary|
+---------------------+---+----------+-------+----+------+
|[{Pune, Maharashtra}]|25 |11        |satish |1201|5000  |
|[{Pune, Maharashtra}]|28 |21        |krishna|1202|6000  |
|[{Pune, Maharashtra}]|39 |31        |amith  |1203|7000  |
|[{Pune, Maharashtra}]|23 |41        |javed  |1204|8000  |
|[{Pune, Maharashtra}]|23 |41        |prudvi |1205|9000  |
+---------------------+---+----------+-------+----+------+



In [37]:
"""
Problem 1: Given 2 Datasets employee.json and dept.json
We need to calculate the count of employees against each department. Use
Structured API’s.   header : depName,deptid,empcount"""
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql import functions as F

join_condn = deptDf1.deptid_dept == empDf1.deptid_emp
joinedDf = deptDf1.join(empDf1,join_condn,"LEFT")
joinedDf.show(truncate = False)

result = joinedDf.select(
    F.col("deptName").alias("deptName"),
    F.col("deptid_dept").alias("deptid"),
    F.count("id").over(Window.partitionBy("deptid_dept").orderBy("deptid_dept")).alias("empCount")
).distinct().orderBy("deptid_dept")


result1 = joinedDf.selectExpr("deptName","deptid_dept as deptid",
                              "count(id)over(partition by deptid_dept order by deptid_dept) as empCount1").distinct().orderBy("deptid_dept")

# worked on both ways column object and using Sql Expression  final results
result.show()
result1.show()


+---------+-----------+---------------------+----+----------+-------+----+------+
|deptName |deptid_dept|address              |age |deptid_emp|empname|id  |salary|
+---------+-----------+---------------------+----+----------+-------+----+------+
|IT       |11         |[{Pune, Maharashtra}]|25  |11        |satish |1201|5000  |
|HR       |21         |[{Pune, Maharashtra}]|28  |21        |krishna|1202|6000  |
|Marketing|31         |[{Pune, Maharashtra}]|39  |31        |amith  |1203|7000  |
|Fin      |41         |[{Pune, Maharashtra}]|23  |41        |prudvi |1205|9000  |
|Fin      |41         |[{Pune, Maharashtra}]|23  |41        |javed  |1204|8000  |
|Admin    |51         |null                 |null|null      |null   |null|null  |
+---------+-----------+---------------------+----+----------+-------+----+------+

+---------+------+--------+
| deptName|deptid|empCount|
+---------+------+--------+
|       IT|    11|       1|
|       HR|    21|       1|
|Marketing|    31|       1|
|      Fin|